In [1]:
!nvidia-smi

Fri Feb 12 18:44:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:01:01.0 Off |                    0 |
| N/A   24C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#checking the availability of GPU
print(len(tf.config.experimental.list_physical_devices("GPU")))

1


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Dense,Flatten,Layer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.utils import np_utils
import tensorflow_datasets as tfds
import tensorflow as tf

In [5]:
x_train , y_train = tfds.as_numpy(tfds.load('horses_or_humans',split = 'train', batch_size = -1, as_supervised = True))

In [6]:
print(x_train.shape)

(1027, 300, 300, 3)


In [7]:
x_test , y_test = tfds.as_numpy(tfds.load('horses_or_humans',split = 'test', batch_size = -1, as_supervised = True))

In [8]:
print(x_test.shape)

(256, 300, 300, 3)


In [9]:
x_train = x_train/255
x_test = x_test/255 

In [10]:
x_train = tf.image.resize(x_train,[224,224])
x_test = tf.image.resize(x_test,[224,224])

In [11]:
x_train.shape

TensorShape([1027, 224, 224, 3])

In [23]:
y_train = np_utils.to_categorical(y_train,2)
y_test = np_utils.to_categorical(y_test,2)

In [13]:
#loading the vgg model and include_top = True will keep the dense layers which is after the bottleneck layer as it is
vgg = VGG16(include_top = True, weights = 'imagenet')

In [14]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
#we don't want our vgg network weights to get retrained we want the weights as it is
for layer in vgg.layers:
    layer.trainable = False

In [25]:
#taking the vgg input from first input layer and till the last dense layer and adding our own output layer based on our custom model
vgg_input = vgg.layers[0].input
vgg_output = vgg.layers[-2].output
predictions = Dense(2)(vgg_output)

In [17]:
#creating our new model
model = Model(inputs = vgg_input,outputs = predictions)

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [30]:
model.fit(x_train,y_train ,epochs = 5)

Epoch 1/5
33/33 [==============================] - 5s 156ms/step - loss: 6.1735 - accuracy: 0.5881
Epoch 2/5
33/33 [==============================] - 4s 134ms/step - loss: 7.9152 - accuracy: 0.5910
Epoch 3/5
33/33 [==============================] - 4s 134ms/step - loss: 7.9152 - accuracy: 0.5239
Epoch 4/5
33/33 [==============================] - 4s 133ms/step - loss: 7.9152 - accuracy: 0.5229
Epoch 5/5
33/33 [==============================] - 4s 134ms/step - loss: 7.9152 - accuracy: 0.5229


In [31]:
model.evaluate(x_test,y_test)

8/8 [==============================] - 1s 120ms/step - loss: 7.7125 - accuracy: 0.5391


[7.7124738693237305, 0.5390625]